# 多进程与多线程<br>
为了充分挖掘计算机的运算能力,采用多进程和多线程的方式进行运算<br>
多进程与多线程的方法或函数的**首字母都需要大写!!**
## 添加进程
**td.Thread**是一个对象 一个属性: __init__(self, group=None, target=None, name=None, args=())<br>
**mp.Process**用法和它高度类似

In [9]:
import multiprocessing as mp
import threading  as td
#创建线程,进程
def job(a,d):
    print('a job')
#help(mp.Process())
#help(td.Thread())
td1=td.Thread(target=job,args=(1,2))#target是调用的对象,args存放参数
mp1=mp.Process(target=job,args=(1,2))
#开始多线程和多进程
td1.start()
mp1.start()
#连接线程和进程
td1.join()
mp1.join()

a job


In [12]:
import multiprocessing as mp
def plus(a,b):
    print(a+b)
if __name__=='__main__':
    mp2=mp.Process(target=plus,args=(10,-3))
    mp2.start()
    mp2.join()
#没有输出属于正常现象,在treminal下会有输出

### 在queue中存放结果<br>
多进程在处理过程中是没有返回值的,我们习惯上用queue来存放程序运行的结果

In [ ]:
import multiprocessing as mp
def job(Q):
    L0=0
    for i in range(100):
        L0+=i
    Q.put(L0)
if __name__=='__main__':
    Q=mp.Queue()
    #定义p1,p2两个线程,同时处理job
    p1=mp.Process(target=job,args=(Q,))
    p2=mp.Process(target=job,args=(Q,))
    p1.start()
    p2.start()
    p2.join()
    p1.join()
    Sum=Q.get()#两个进程同时操作的和
    print(Sum)

## 多线程&多进程横评<br>
运行时间通常是 多进程 < 普通 < 多线程，可见**mutliprocessing**效率最高。

In [ ]:
import multiprocessing as mp
import threading as td
import time  
def job(Qu):
    result=0
    for i in range(5000000):
        result+=i+i**2
    Qu.put(result)
def normal():
    result=0
    for i in range(5000000):
        result+=i+i**2
    return result

if __name__=='__main__':
    startp=time.time()
    Qup=mp.Queue()
    p1=mp.Process(target=job,args=(Qup,))
    p2=mp.Process(target=job,args=(Qup,))
    p1.start()
    p2.start()
    p1.join()
    p2.join()
    res1=Qup.get()
    res2=Qup.get()
    print(res1)
    print(time.time()-startp)
    
    startt=time.time()
    Qut=mp.Queue()
    t1=td.Thread(target=job,args=(Qut,))
    t2=td.Thread(target=job,args=(Qut,))
    t1.start()
    t2.start()
    t1.join()
    t2.join()
    res1=Qut.get()
    res2=Qut.get()
    print(res1)
    print(time.time()-startt)
    
    startn=time.time()
    print(normal())
    print(time.time()-startn)

## 进程池pool<br>
Pool可以由CPU主动进行多核处理<br>
进程池可以获取返回值 pool.map()<br>
还可以自定义核心数为n:  pool=mp.Pool(processes=n)

In [ ]:
def job(x):
    return x*x
def multi():
    pool=mp.Pool()
    result=pool.map(job,range(10))
    print(result)
if __name__=='__main__':
    multi()

**pool.apply_async()**也可以用于返回值()<br>
它只能单值返回,好在它是可迭代的,用 _.get()_ 方法获取结果

In [ ]:
def job(x):
    return x*x
#并不一定要写成函数的形式,也可以直接在 __main__ 里边
def multi():
    pool=mp.Pool()
    result=pool.apply_async(job,(6,)) #传入值注意是可迭代的，所以后需要加逗号
    print(result.get())
    result=[pool.apply_async(job,(i,)).get() for i in range(10)] 
    print(result)
    return result
    
if __name__=='__main__':
    multi()

## 共享内存<br>
多进程之间的信息需要交互与共享,因此有**shared memory**<br>
一个multiprocessing中蛮有意思的功能.[官方格式](https://docs.python.org/3/library/array.html)

In [ ]:
import multiprocessing as mp

#shared Value 只能存放一个数字
data1=mp.Value('i',0) 
data2=mp.Value('b',67) #data2是'C'
print(data1)
#shared Array 只能存放一维数组
lis1=mp.Array('d',[2.4,6,7.2,0.9])
lis2=mp.Array('L',[45645465456,4565456465])
print(lis2[0])

## 进程锁<br>
***mp.Lock()***<br>进程锁的存在是为了仿真多进程处理时轮流抢占内存,加了进程锁可以让两个进程分立运行.

In [ ]:
import multiprocessing as mp
import time
def job(val,step,lock):
    lock.acquire()  #上锁
    for i in range(5):
        val.value+=step
        print(val.value)
        time.sleep(0.1)
    lock.release()  #开锁

def uselock():
    val=mp.Value('i',0)  #共享的数据
    lock=mp.Lock()  #建立进程锁
    p1=mp.Process(target=job,args=(val,1,lock))
    p2=mp.Process(target=job,args=(val,3,lock))
    p1.start()
    p2.start()
    p1.join()
    p2.join()

if __name__=='__main__':
    uselock()
#没有输出
